# Figures for Chapter on SAR

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import os
from pathlib import Path
pad = Path(os.getcwd())
if pad.name == "Figures":
    pad_correct = pad.parent
    os.chdir(pad_correct)
from functions.plotting_functions import plot_discrete_raster
write_fig = True

## Landuse and SAR 2D image

In [ ]:
s1_xr = xr.open_dataset('data/s0_OpenEO/S0_zwalm_landuse.nc', decode_coords= 'all')
s1_xr = s1_xr.chunk({'t':20})
s1_xr['VV_db'] = 10 * np.log10(s1_xr['VV'])
s1_xr

In [ ]:
colormap = colors.ListedColormap(
            [
                "#000000",
                "#387b00",
                "#64cf1b",
                 "#a47158",
                "#00bfff",
            ]
        )
labels = [
    'Urban',
    'Forest',
    'Pasture',
    'Agriculture',
    'Water'
]

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (11.5,5),constrained_layout = True)
s1_xr['VV_db'].isel(t=0).plot(ax = axes[0],  cbar_kwargs = {'label':r'$\sigma^0$ [dB]'}, rasterized = True)
axes[0].set_aspect('equal','box')
axes[0].set_title('(a)')
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
# s1_xr['landuse'].plot(ax = axes[1])
# axes[1].set_aspect('equal','box')
fig, ax_1 = plot_discrete_raster(
    fig, axes[1], s1_xr['landuse'].values, bounds = list(s1_xr.rio.bounds()), 
    labels = labels, cmap = colormap, rasterized = True
)
ax_1.set_aspect('equal','box')
ax_1.set_xlabel('x [m]')
ax_1.set_ylabel('y [m]')
ax_1.set_title('(b)')
figures_SAR_path = Path("Figures/Figures_chapter_SAR")
if not os.path.exists(figures_SAR_path):
    os.makedirs(figures_SAR_path)
if write_fig:
    fig.savefig(figures_SAR_path/'sigma0_landuse_example.pdf',format = 'pdf')

In [ ]:
s1_xr.isel(t=0)['t'].values

In [ ]:
list(s1_xr.rio.bounds())

## Timeseries

In [ ]:
s1_pd = pd.read_csv('data/s0_OpenEO/s1_timeseries.csv',index_col ='t', parse_dates=True)
s1_pd.head()

In [ ]:
s1_pd_ascending = s1_pd[s1_pd['Orbitdirection'] == 'ascending']
s1_pd_descending = s1_pd[s1_pd['Orbitdirection'] == 'descending']
cols = s1_pd.columns
VV_cols = cols[0:5]
names_legend = [name[2:] for name in VV_cols]
print(names_legend)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize = (10.5,7),constrained_layout = True)

#ascending
s1_pd_ascending[VV_cols].plot.line(ax = ax1)
ax1.set_title('Ascending')
ax1.set_ylabel(r'$\sigma^0_{VV}$ [dB]')
ax1.legend(names_legend, loc = 'upper right')
ax1.set_xlabel('Timestamp')

#descending
s1_pd_descending[VV_cols].plot.line(ax = ax2)
ax2.set_ylabel(r'$\sigma^0_{VV}$ [dB]')
ax2.set_title('Descending')
ax2.legend(names_legend, loc = 'upper right')
ax2.set_xlabel('Timestamp')
if write_fig:
    fig.savefig(figures_SAR_path/'sigma0_timeseries.pdf',format = 'pdf')